# Higher-order functions

In this notebook, we will see that higher-order functions (HOFs) are essential modularity devices, and we will introduce the most common higher-order functions that operate over many different data structures.

In [1]:
import $ivy.`org.scalatest::scalatest:3.0.8`
import _root_.org.scalatest._

import $ivy.$                               

import _root_.org.scalatest._

### References

[__Programming in Scala, 
A comprehensive step-by-step guide__](https://www.artima.com/shop/programming_in_scala_3ed) Third Edition.
by Martin Odersky, Lex Spoon, and Bill Venners. 

- Chapter 6. Functional objects
- Chapter 8. Functions and closures
- Chapter 16. Working with Lists
- Chapter 23. For Expressions Revisited

[__Functional programming in Scala__](https://www.manning.com/books/functional-programming-in-scala), by Paul Chiusano and Runar Bjarnason.

- Chapter 3. Functional data structures

[__Functional programming simplified__](https://alvinalexander.com/downloads/fpsimplified-free-preview.pdf), by Alvin Alexander.

- Chapter 22. Functions Are Variables, Too
- Chapter 23. Using Methods As If They Were Functions
- Chapter 24. How to Write Functions That Take Functions as Input Parameters
- Chapter 25. How to Write a ‘map’ Function
- Chapter 27. Functions Can Have Multiple Parameter Groups
- Chapter 28. Partially-Applied Functions (and Currying)

## `FoldRight`: divide and conquer

HOFs shine when the time comes to break monoliths. For instance, let's consider the following two functions:

In [3]:
def sum(list: List[Int]): Int = 
    list match {
        case Nil => 0
        case head :: tail =>
            val tailSol: Int = sum(tail)
            tailSol + head
    }

defined function sum

In [4]:
def multiply(list: List[Int]): Int = 
    list match {
        case Nil => 1
        case head :: tail =>
            val tailSol: Int = multiply(tail)
            tailSol * head 
    }

defined function multiply

In [5]:
def someEven(list: List[String]): Boolean = 
    list match {
        case Nil => false
        case head :: tail =>
            val tailSol: Boolean = someEven(tail)
            head.length % 2 == 0 || tailSol
    }

defined function someEven

In [6]:
someEven(List("a", "abc", "b", "abddd")) == false
someEven(List("a", "aa")) == true

res5_0: Boolean = true
res5_1: Boolean = true

These functions clearly share a common logic: their only differences are the value which is returned when the list is empty, and the function used to combine numbers (`+` and `*`, respectively). We can abstract away these differences and arrive to a more generic function which encodes that common logic:

In [10]:
def dyv[A, B](list: List[A])(atomic: B)( 
                             combine: (A, B) => B ): B = 
    list match {
        case Nil => atomic : B
        case head :: tail =>
            val tailSol: B = dyv(tail)(atomic)(combine)
            combine(head, tailSol) : B
    }

defined function dyv

which allows us to re-define in a modular way the `sum` and `multiply` functions:

In [11]:
def sum(list: List[Int]): Int = 
    dyv[Int, Int](list)(
        0 : Int)( 
        (head: Int, tailSol: Int) => 
            head + tailSol : Int
    )

defined function sum

In [ ]:
def multiply(list: List[Int]): Int = 
    dyv[Int, Int](list)(
        1 : Int , 
        (head: Int, tailSol: Int) => 
            head * tailSol : Int
    )

In [13]:
def multiply(list: List[Int]): Int = 
    dyv(list)(1)(_ * _)

defined function multiply

In [9]:
def someEven(list: List[String]): Boolean = 
    dyv[String, Boolean](list)(false,
        (head: String, tailSol: Boolean) => 
            head.length % 2 == 0 || tailSol
    )

defined function someEven

But we don't need to constrain ourselves to integers. In its generic version, the `combine` function is actually the `foldRight` higher-order function (for `List`s):

Graphically, we can explain the behaviour of `foldRight` as follows: 

![with elements](../images/foldRight.1.svg)

(all images credit to [Scala Visual Reference](https://superruzafa.github.io/visual-scala-reference/))

We can understand `foldRight` as an implementation of the divide-and-conquer design pattern: first, divide your problems in subproblems; second, solve these sub-problems; last, compose their solutions to obtain the solution to the overall problem. If the sub-problems are simple enough they can be solved directly; otherwise, they are solved recursively. In the case of lists:
- The problem is obtaining a value of type `B` for a given list.
- The only sub-problem corresponds to the tail of that list. 
- The arguments of the `foldRight` function tell us how to obtain the solution for the empty list (the atomic problem which can not be further decomposed), and how to obtain the solution from the solution to the sub-problem.

From this perspective, the implementation of `sum` and `multiply` is exactly the same as before when we used the function `combine`, although we can explain them differently:

In [14]:
def sum(list: List[Int]): Int = 
    list.foldRight(0)((head, tailSol) => head + tailSol)

defined function sum

In [15]:
def sum(list: List[Int]): Int = 
    list.foldRight(0)(_ + _)

defined function sum

In [ ]:
def multiply(list: List[Int]): Int = 
    ???

#### Improved type-inference

Technically, you may have noticed that we have to give extra type information in the invocations to `foldRight`. In fact, the following code doesn't compile. Check it yourself!

In [ ]:
/*
def multiply(list: List[Int]): Int = 
    foldRight(list)(1, (a, b) => a * b)
*/

In order to help the Scala compiler to infer the type parameters of the `foldRight` function, we need to change its signature a little bit: 

By splitting the second parameter list, we allow the Scala compiler to infer the type of `B`, before it analyses the type of the `cons` argument. Now this works:

#### HOFs in the Scala API

The `foldRight` HOF, as all the HOFs that we will explain in this notebook, are actually part of the standard Collections library of Scala. Typically, they are implemented as member methods of the corresponding collection class. For lists, the [Scala API](https://www.scala-lang.org/api/2.13.1/scala/collection/immutable/List.html) offers something like this:

In [ ]:
object Standard{
    class List[A]{
        def foldRight[B](b: B)(f: (B, A) => B): B = ???
    }
}

So, invocations of the standard `foldRight` do not receive the list as argument, but are common method invocations on some `List` instance:

Normally, we will use standard HOFs instead of our home-made versions, once they are explained. 

## The Hall of Fame of HOFs

Besides `compose`, `andThen` and `foldRight`, there are other famous higher-order functions which work great as modularity devices: 

In [ ]:
def foldLeft[A, B](list: List[A])(b: B)(f: (B, A) => B): B = ???
def filter[A](list: List[A])(f: A => Boolean): List[A] = ???
def map[A, B](list: List[A])(f: A => B): List[B] = ???
def flatMap[A, B](list: List[A])(f: A => List[B]): List[B] = ???

But most of these functions do not only work for `List`, they work also for `Option`, `Either[A, ?]`, and many other data structures. For instance:

In [ ]:
def fold[A, B](opt: Option[A])(none: B)(some: A => B): B = ???
def filter[A](opt: Option[A])(f: A => Boolean): Option[A] = ???
def map[A, B](opt: Option[A])(f: A => B): Option[B] = ???  
def flatMap[A, B](opt: Option[A])(f: A => Option[B] ): Option[B] = ???

In [ ]:
def fold[A, B, C](opt: Either[A, B])(left: A => C, right: B => C): C = ???
def filter[A, B](opt: Either[A, B])(f: B => Boolean): Either[A, B] = ???
def map[A, B, C](opt: Either[A, B])(f: B => C): Either[A, C] = ???  
def flatMap[A, B, C](opt: Either[A, B])(f: B => Either[A, C] ): Either[A, C] = ???

The `fold` function is the catamorphism for `Option` and `Either[A, ?]`, in the same way than `foldRight` is the catamorphism for lists (we will see catamorphisms later on). 

## FoldLeft: a better loop

The `foldLeft` HOF is the functional way to implement common imperative algorithms. For instance:

In [ ]:
def length[A](list: List[A]): Int = {
    var out: Int = 0
    for (e <- list)
        out += 1
    out
}

In [15]:
def reverse[A](list: List[A]): List[A] = {
    var out: List[A] = ??? 
    for (e <- list)
        out = ???(e, out)
    out
}

cmd15.sc:4: Nothing does not take parameters
        out = ???(e, out)
                 ^Compilation Failed

: 

In [31]:
def reverse[A](list: List[A]): List[A] = {
    def aux(out: List[A], l: List[A]): List[A] = 
        l match {
            case Nil => out
            case e :: tail => 
                aux(e :: out, tail)
        }
    
    aux(Nil, list)
}

defined function reverse

In [16]:
def reverse[A](list: List[A]): List[A] = {
    var out: List[A] = Nil
    for (e <- list)
        out = e :: out
    out
}

defined function reverse

In [17]:
reverse(List(1,2,3,4,5))

res16: List[Int] = List(5, 4, 3, 2, 1)

The following implementation abstracts away the differences in the above functions, and declares two additional parameters: the initial value of a mutable variable, and the update function executed in every step of the loop.

In [20]:
def iterative[A, B](list: List[A])(initial: B)(
                                update: (B, A) => B
): B = {
    var out: B = initial
    for (e <- list)
        out = update(out, e)
    out
}

defined function iterative

In [32]:
def iterative[A, B](list: List[A])(initial: B, 
                                update: (B, A) => B): B = {
    def aux(out: B, l: List[A]): B = 
        l match {
            case Nil => out
            case e :: tail => 
                aux(update(out, e), tail)
        }
    
    aux(initial, list)
}

defined function iterative

In [33]:
def iterative[A, B](list: List[A])(initial: B, 
                                update: (B, A) => B): B = 
    list match {
        case Nil => initial
        case e :: tail => 
            iterative(tail)(update(initial, e),update)
    }

defined function iterative

In [20]:
def iterative[A, B](list: List[A])(initial: B)(
                                update: (B, A) => B
): B = {
    var out: B = initial
    for (e <- list)
        out = update(out, e)
    out
}

defined function iterative

Similarly, the `foldLeft` function is typically used where a tail-recursive function is needed. Its recursive implementation is naturally tail-recursive:

In [ ]:
//@annotation.tailrec
def foldLeft[A, B](list: List[A])(out: B)(update: (B, A) => B): B =
    ???

Using `foldLeft` we can implement functions at a higher-level of abstraction, i.e. using constructs which are nearer to the problem-domain, without caring about mutable variables, tail-safe recursion, etc. Its use also leads to very concise (and readable!) implementations. For instance, these are (almost) one-liner implementations of some of the functions of the last notebook on recursive functions:

In [21]:
def length[A](list: List[A]): Int = 
    iterative[A, Int](list)(
        ??? : Int)( 
        (out: Int, e: A) => ??? : Int)

defined function length

In [22]:
def length[A](list: List[A]): Int = 
    iterative[A, Int](list)(
        0 : Int)(
        (out: Int, _: A) => out + 1 : Int)

defined function length

In [24]:
def length[A](list: List[A]): Int = 
    iterative(list)(0)((out, _) => out + 1)

defined function length

In [ ]:
def sum(list: List[Int]): Int = 
    ???

In [25]:
def reverse[A](list: List[A]): List[A] = 
    iterative(list)(
        Nil : List[A])(
        (out : List[A], e: A) => 
            e :: out : List[A])

defined function reverse

In [25]:
def reverse[A](list: List[A]): List[A] = 
    iterative(list)(
        Nil : List[A])(
        (out : List[A], e: A) => 
            e :: out : List[A])

defined function reverse

In [27]:
def reverse[A](list: List[A]): List[A] = 
    list.foldLeft[List[A]](Nil)( (out, e) => e :: out)

defined function reverse

In [29]:
def reverse[A](list: List[A]): List[A] = 
    list.foldLeft(List[A]())( (out: List[A], e: A) => e :: out)

defined function reverse

In [30]:
def reverse[A](list: List[A]): List[A] = 
    list.foldLeft(List[A]())( (out, e) => e :: out)

defined function reverse

But we may also use the `foldRight` function to implement them:

In [ ]:
def length[A](list: List[A]): Int = 
    ???

In [ ]:
def sum(list: List[Int]): Int = 
    ???

In [ ]:
def reverse[A](list: List[A]): List[A] = 
    ???

Functionally, both versions are equivalent, but the ones using `foldLeft` are more efficient. In particular, the `foldRight` version of `reverse` has cuadratic complexity. 

#### Implementing `foldRight` with `foldLeft`

The implementation that we gave for `foldRight` was not tail-recursive, so this will blow up the stack:

In [ ]:
foldRight[Int, Int](List.fill(100000)(1))(0)((_,_)=> 0)

In order to obtain a better implementation, we may first reverse the list and use `foldLeft` as follows: 

In [ ]:
def foldRightTR[A, B](list: List[A])(nil: B)(cons: (A, B) => B): B = 
    ???

This implementation works right with big lists:

In [ ]:
// foldRightTR[Int, Int](List.fill(100000)(1))(0)((_,_)=> 0)

#### Implementing `foldLeft` with `foldRight`

This is also possible, though challenging. We start from the tail-recursive implementation of `foldLeft`. First, we rearrange the arguments so that it now returns a function instead of a plain value of type `B`. 

In [ ]:
def foldLeft[A, B](list: List[A])(out: B)(update: (B, A) => B): B =
    list match {
        case Nil => out
        case head :: tail => 
            foldLeft(tail)(update(out, head))(update)
    }

This was in order to better expose the recursion pattern of `foldRight`. The solution is now obtained simply by noticing that `foldLeft(tail)(update)` is the solution for the tail:

In [ ]:
def foldLeftAux[A, B](list: List[A])(update: (B, A) => B): B => B =
    ???

The original signature can then be implemented as follows: 

In [ ]:
def foldLeft[A, B](list: List[A])(out: B)(update: (B, A) => B): B =
    ???

## The `map` HOF

In [3]:
def lengths(l: List[String]): List[Int] = 
    l match {
        case Nil => Nil
        case head :: tail => 
            val tailSol: List[Int] = lengths(tail)
            head.length :: tailSol
    }

defined function lengths

In [6]:
def lengths(l: List[String]): List[Int] = 
    l.foldRight(Nil : List[Int])(
        (head: String, tailSol: List[Int]) => 
            head.length :: tailSol : List[Int]
    )

defined function lengths

In [7]:
lengths(List("", "a", "BBB", "sddsdsdfsf")) == 
        List(0,  1,    3,     10)

lengths(List()) ==
        List()

res6_0: Boolean = true
res6_1: Boolean = true

In [8]:
def positives(l: List[Int]): List[Boolean] = 
    l match {
        case Nil => Nil
        case head :: tail => 
            val tailSol: List[Boolean] = positives(tail)
            (head > 0) :: tailSol
    }

defined function positives

In [12]:
def positives(l: List[Int]): List[Boolean] = 
    //map(l)( head => head > 0)
    map(l)(_ > 0)

defined function positives

In [3]:
def lengths(l: List[String]): List[Int] = 
    l match {
        case Nil => Nil
        case head :: tail => 
            val tailSol: List[Int] = lengths(tail)
            head.length :: tailSol
    }

defined function lengths

In [3]:
def lengths(l: List[String]): List[Int] = 
    l match {
        case Nil => Nil
        case head :: tail => 
            val tailSol: List[Int] = lengths(tail)
            head.length :: tailSol
    }

defined function lengths

In [9]:
def positives(l: List[Int]): List[Boolean] = 
    l.foldRight(List[Boolean]())(
        (head, tailSol) => (head > 0) :: tailSol)

defined function positives

In [14]:
def positives(l: List[Int]): List[Boolean] = 
    map(l)(_ > 0)

defined function positives

In [15]:
def positives(l: List[Int]): List[Boolean] = 
    l.map(_ > 0)

defined function positives

In [10]:
def map[A, B](l: List[A])(f: A => B): List[B] = 
    l match {
        case Nil => Nil
        case head :: tail => 
            val tailSol: List[B] = map(tail)(f)
            f(head) :: tailSol
    }

defined function map

The `map` function is one of the essential HOFs. Its purpose is applying a function to the elements of a data structure, in such a way that the relationships between these elements do not change. The only thing that must be modified is the _content_ of the data structure, not its _shape_. This condition is expressed in the following equations:

1. `map(list)(identity) == list` for all `list: List[A]`
2. `map(map(list)(f))(g) == map(list)(g compose f)` for all `list: List[A]`, `f: A => B`, `g: B => C`

![map](../images/map.svg)

The implementation for lists can be done as follows:

In [ ]:
def map[A, B](list: List[A])(f: A => B): List[B] = 
    ???

The `map` function is polymorphic in `A` and `B`, but we can't pass generic functions (aka polymorphic values) as parameters using `FunctionN`classes (these classes only wrap functions from specific types to specific result types). We may create polymorphic versions of `FunctionN` clases, but in order to keep things simple, we will define the test catalogue for `map` using a specific signature chosen at random (any other may serve as well):

In [ ]:
class TestMap(
    map: List[Int] => (Int => Boolean) => List[Boolean]
) extends FlatSpec with Matchers{
    
    val isEven: Int => Boolean = _ % 2 == 0
    
    "mapping an empty list" should "return an empty list" in {
        map(List())(isEven) shouldBe ???
    }
    
    "mapping an non-empty list" should "only change its content" in {
        map(List(1))(isEven) shouldBe ???
        map(List(1,2))(isEven) shouldBe ???
        map(List(1,2,3))(isEven) shouldBe ???
    }
}

To run the tests we will pass the `map` function for `Ìnt` and `Boolean` types:

In [ ]:
run(new TestMap(map[Int, Boolean]))

The implementation of `map` is really close to the implementation of `foldRight`. Indeed, we can give a more modular implementation using this more basic HOF: 

In [ ]:
/*
def foldRight[A, B](list: List[A])(nil: B)(cons: (A, B) => B): B = 
    list match {
        case Nil => nil
        case head :: tail => cons(head, foldRight(tail)(nil)(cons))
    }
*/

def mapFR[A, B](list: List[A])(f: A => B): List[B] = 
    ???

In [ ]:
run(new TestMap(mapFR))

### `map` for `Option`

The implementation of the `map` function given above works over lists, but we can also map optional values and many other data structures. The implementation for the data structure `Option` must satisfy the following test:

In [16]:
class TestMapOption(
    map: Option[Int] => (Int => Boolean) => Option[Boolean]
) extends FlatSpec with Matchers{
    
    "mapping the `None` value" should "return `None`" in {
        map(None)(_ % 2 == 0) shouldBe None
    }
    
    "mapping a `Some` value" should "only change its content" in {
        map(Some(3))(_ % 2 == 0) shouldBe Some(false)
        map(Some(1))(_ % 2 == 0) shouldBe Some(false)
        map(Some(2))(_ % 2 == 0) shouldBe Some(true)
    }
    
        
}

defined class TestMapOption

The implementation is simple:

In [17]:
def mapOpt[A, B](maybeA: Option[A])(f: A => B): Option[B] = 
    maybeA match {
        case None => None
        case Some(a) => 
            Some(f(a))
    }

defined function mapOpt

In [18]:
run(new TestMapOption(mapOpt))

cmd15$Helper$TestMapOption:
mapping the `None` value
- should return `None`
mapping a `Some` value
- should only change its content


In [19]:
Some(4).map(_ % 2 == 0)

res18: Option[Boolean] = Some(true)

## Filtering elements

In [20]:
def positives(l: List[Int]): List[Int] = {
    var out: List[Int] = Nil
    for (e <- l)
        if (e > 0) out = e :: out
    out.reverse
}

defined function positives

In [33]:
def positives(l: List[Int]): List[Int] = 
    filter(l)(e => e > 0)

defined function positives

In [22]:
positives(List(-1,0,1,3))
positives(List(-1,-2,-3))

res21_0: List[Int] = List(1, 3)
res21_1: List[Int] = List()

In [28]:
"abc"(0) == 'b'
"" != "" && "a"(0) == 'a'

res27_0: Boolean = false
res27_1: Boolean = false

In [29]:
def astrings(l: List[String]): List[String] = {
    var out: List[String] = Nil
    for (e <- l)
        if (e != "" && e(0) == 'a') out = e :: out
    out.reverse
}

defined function astrings

In [32]:
def astrings(l: List[String]): List[String] = 
    filter(l)(e => e != "" && e(0) == 'a')

defined function astrings

In [30]:
astrings(List("b", "ccc", "abb", "aa"))

res29: List[String] = List("abb", "aa")

In [31]:
def filter[A](l: List[A])(pred: A => Boolean): List[A] = {
    var out: List[A] = Nil
    for (e <- l)
        if (pred(e)) out = e :: out
    out.reverse
}

defined function filter

In [34]:
def filter[A](l: List[A])(pred: A => Boolean): List[A] = {
    def aux(out: List[A], l: List[A]): List[A] = 
        l match {
            case Nil => out
            case e :: tail => 
                aux(if (pred(e)) e :: out 
                    else out, tail)
        }
    
    aux(Nil, l).reverse
}

defined function filter

In [35]:
def filter[A](l: List[A])(pred: A => Boolean): List[A] = 
    l.foldLeft(Nil: List[A])(
        (out, e) => if (pred(e)) e :: out 
                    else out
    ).reverse

defined function filter

In [36]:
filter(List(1,2,3,4))(_ % 2 == 0)

res35: List[Int] = List(2, 4)

Unlike `map`, the `filter` HOF allows us to change the shape of the data structure, removing those elements that do not satisfy a given predicate. 

![filter](../images/filter.svg)

In [ ]:
def filterR[A](list: List[A])(predicate: A => Boolean): List[A] = 
   ???

In [ ]:
class TestFilterList(
    filter: List[Int] => (Int => Boolean) => List[Int]
) extends FlatSpec with Matchers{
    
    "filter" should "work" in {
        filter(List())(_ % 2 == 0) shouldBe List()
        filter(List(1))(_ % 2 == 0) shouldBe List()
        filter(List(1,3,5))(_ % 2 == 0) shouldBe List()
        filter(List(2,4,6))(_ % 2 == 0) shouldBe List(2,4,6)
    }
}

In [ ]:
run(new TestFilterList(filter))

Using `foldRight` we can get a more modular implementation: 

In [ ]:
def filterFR[A](list: List[A])(predicate: A => Boolean): List[A] = 
    ???

In [ ]:
run(new TestFilterList(filterFR))

### Filtering optional values

We only have a possible value, so implementing filtering is easy in this case:

In [ ]:
class TestFilterOption(
    filter: Option[Int] => (Int => Boolean) => Option[Int]
) extends FlatSpec with Matchers{
    
    "filter" should "work" in {
        filter(None)(_ % 2 == 0) shouldBe ???
        filter(Some(1))(_ % 2 == 0) shouldBe ???
        filter(Some(2))(_ % 2 == 0) shouldBe ???
    }
}

In [ ]:
def filter[A](maybeA: Option[A])(predicate: A => Boolean): Option[A] = 
    ???

In [ ]:
run(new TestFilterOption(filter))

## FlatMapping data structures

Let's consider a paragraph represented as a list of sentences, where each sentence is in turn modelled as a string made of _words_ separated by blank spaces. We want to obtain a list of all the words in each sentence. 

In [38]:
class TestWords(
    words: List[String] => List[String]
) extends FlatSpec with Matchers{
            
    val paragraph1 = List(
        "En un lugar",
        "de la Mancha", 
        "de cuyo nombre no",
        "quiero acordarme")
    
    "words" should "work" in {
        words(paragraph1) shouldBe 
            List("En", "un", "lugar", "de", "la", "Mancha",
                "de", "cuyo", "nombre", "no", "quiero",
                "acordarme")
    }
}

defined class TestWords

We may try to map the paragraph with a function that `split`s  each sentence into the words they are made of:

In [40]:
"en un lugar".split(" ").toList

res39: List[String] = List("en", "un", "lugar")

In [52]:
val paragraph: List[String] = 
    List("en un lugar", 
            "de la Mancha")

paragraph.map((line: String) => 
                  line.split(" ").toList : List[String]) == 
    List(List("en", "un", "lugar"), 
         List("de", "la", "Mancha"))

paragraph.map(line => line.split(" ").toList)
        .flatten

paragraph.flatMap(line => line.split(" ").toList)

paragraph.flatMap(_.split(" ").toList)

paragraph.flatMap(_.split(" "))


paragraph: List[String] = List("en un lugar", "de la Mancha")
res51_1: Boolean = true
res51_2: List[String] = List("en", "un", "lugar", "de", "la", "Mancha")
res51_3: List[String] = List("en", "un", "lugar", "de", "la", "Mancha")
res51_4: List[String] = List("en", "un", "lugar", "de", "la", "Mancha")
res51_5: List[String] = List("en", "un", "lugar", "de", "la", "Mancha")

In [46]:
def concatenate[A](l: List[List[A]]): List[A] = 
    l.foldRight( Nil : List[A])(
        (head: List[A], tailSol: List[A]) => 
            head ++ tailSol
        )

defined function concatenate

In [47]:
concatenate(List(List(1), List(2,3,4), List(5,6), List()))

res46: List[Int] = List(1, 2, 3, 4, 5, 6)

In [48]:
List(List(1), List(2,3,4), List(5,6), List()).flatten

res47: List[Int] = List(1, 2, 3, 4, 5, 6)

but then we obtain a _list of lists_ of strings, not a list of plain strings. The solution is not far though: we have just to concatenate all the lists and we obtain what we need. The function `flatten` performs this concatenation:

In [ ]:
def flatten[A](listOflists: List[List[A]]): List[A] = 
    ???

The function `flatten` is actually provided by the Scala API. So, the implementation of the `words` function is as follows:

In [ ]:
def words(paragraph: List[String]): List[String] = 
    ???

In [ ]:
run(new TestWords(words))

This combination of the HOF `map` and the function `flatten` is so common, that it has been given a proper name: `flatMap`. 

defined function flatMap

In [57]:
def map[A, B](    list: List[A])(f: A =>      B ): List[B] = 
    ???

def flatMap[A, B](list: List[A])(f: A => List[B]): List[B] = 
    // list.map(f).flatten
    list.foldRight(Nil: List[B]){
        (head: A, tailSol: List[B]) => 
            f(head) ++ tailSol : List[B]
    }


defined function map
defined function flatMap

![filter](../images/flatMap.svg)

Using `flatMap` the word function is implemented even more easily:

In [ ]:
def words(paragraph: List[String]): List[String] = 
    ???

### FlatMapping optional values

In [ ]:
def flatMap[A, B](maybeA: Option[A])(cont: A => Option[B]): Option[B] = 
    ???

In [ ]:
class TestFlatMapOption(
    flatMap: Option[Int] => (Int => Option[Boolean]) => Option[Boolean]
) extends FlatSpec with Matchers {
    
    def isPositiveEven(i: Int): Option[Boolean] = 
        if (i>=0) Some(i % 2 == 0)
        else None
    
    "flatMap" should "work" in {
        flatMap(None)(isPositiveEven) shouldBe ???
        flatMap(Some(5))(isPositiveEven) shouldBe ???
        flatMap(Some(-5))(isPositiveEven) shouldBe ???
        flatMap(Some(0))(isPositiveEven) shouldBe ???
        flatMap(Some(4))(isPositiveEven) shouldBe ???      
    }
}

In [ ]:
run(new TestFlatMapOption(flatMap))

## All together: a modularity problem

As before, we start from a list of sentences, but now we want to take care of the possible extra spaces between each word. Also, we want to obtain not the words themselves but their lengths. So, the specification of our problem is a function `lengths`:

In [58]:
class TestLengths(
    lengths: List[String] => List[Int]
) extends FlatSpec with Matchers{
            
    val paragraph1 = List(
        "En un  lugar",
        "de  la Mancha ", 
        "de cuyo nombre no",
        "quiero        acordarme")
    
    "lengths" should "work" in {
        lengths(paragraph1) shouldBe 
            List(2,2,5,2,2,6,2,4,6,2,6,9)
    }
}

defined class TestLengths

We may try to implement this function imperatively using mutable variables:

In [60]:
"en   un   lugar  ".split(" ")

res59: Array[String] = Array("en", "", "", "un", "", "", "lugar")

In [61]:
def lengthsM(paragraph: List[String]): List[Int] = {
    var out: List[Int] = Nil
    for (line <- paragraph){
        val words: Array[String] = line.split(" ")
        for (word <- words)
            if (word != "")
                out = word.length :: out
    }
    out.reverse
}


defined function lengthsM

In [67]:
def lengthsM(paragraph: List[String]): List[Int] = 
    paragraph.flatMap(_.split(" "))
        .filter(_ != "")
        .map(_.length)


defined function lengthsM

In [65]:
List("en   un  lugar", "de la  mancha")
    .flatMap(_.split(" "))
    .filter(word => word != "")
    .map(word => word.length)

res64: List[Int] = List(2, 2, 5, 2, 2, 6)

In [66]:
List("en   un  lugar", "de la  mancha")
    .flatMap(_.split(" "))
    .filter(_ != "")
    .map(_.length)

res65: List[Int] = List(2, 2, 5, 2, 2, 6)

and this works, of course: 

In [62]:
run(new TestLengths(lengthsM))

cmd57$Helper$TestLengths:
lengths
- should work


but can we do it better? Yes, we can! Using HOFs we can get a more concise, understandable and reliable version, with a great level of reuse!

In [ ]:
def lengthsHOF(paragraph: List[String]): List[Int] = 
    ???

In [ ]:
run(new TestLengths(lengthsHOF))

This solution contrasts with the mutable version in several respects:
- It's more **modular**, i.e. it's made of coarse-grained components: the HOFs `flatMap`, `filter` and `map`. The mutable version builds instead upon fine-grained components: `var`s, and `for` loops.
- The HOF components `flatMap`, etc., are actually very generic and domain-independent, and are typically part of standard libraries, so the level of **reuse** and **reliability** of the HOF-based solution is very high.
- The HOF-based solution is more **understandable**: it models the solution to the problem in terms of standard _transformations_ which are composed together using plain function composition. Moreover, these transformations are also at the right level of abstraction, i.e. it's natural to specify the solution to the problem in terms of flatMap, filter and map.

In [71]:
def sum(l: List[(Int, Int)]): List[Int] = 
    l.map((t: (Int, Int)) => t._1 + t._2 : Int)

defined function sum

In [74]:
def concat(l: List[(String, String)]): List[String] = 
    l.map((t: (String, String)) => t._1 + t._2 : String)

defined function concat

In [78]:
def map2[A, B, C](l: List[(A, B)])(f: (A, B) => C): List[C] = 
    l.map((t: (A, B)) => f(t._1, t._2))

defined function map2

In [75]:
sum(List((1,2), (3,4), (5,6))) == List(3, 7, 11)

res74: Boolean = true

In [76]:
concat(List(("a","b"), ("aa","ab"))) == 
       List("ab", "aaab")

res75: Boolean = true

### For-comprehensions

The combination of `map`, `flatMap` and `filter` HOFs is so common, that Scala has a special syntax for them: for-comprehensions.

In [ ]:
def lengthsFC(paragraph: List[String]): List[Int] = 
    ???

In [ ]:
run(new TestLengths(lengthsFC))